<a href="https://colab.research.google.com/github/VOX304/BasicChatbot/blob/main/Preprocessing/TablePrep/PdfImgTable_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python packages

In [ ]:
!pip install -q ultralytics pdf2image torch torchvision
!pip install PyMuPDF pandas img2table

In [ ]:
!pip install -q paddleocr paddlepaddle paddlepaddle-gpu

In [ ]:
!pip uninstall -y opencv-contrib-python opencv-python
!pip uninstall -y opencv-contrib-python-headless
!pip install opencv-contrib-python

Found existing installation: opencv-contrib-python 4.11.0.86
Uninstalling opencv-contrib-python-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-4.11.0.86
Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
Found existing installation: opencv-contrib-python-headless 4.11.0.86
Uninstalling opencv-contrib-python-headless-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-headless-4.11.0.86
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (69.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddleocr 2.10.0 requires opencv-python, which is not installed.


In [ ]:
!pip install opencv-contrib-python-headless

  Using cached opencv_contrib_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_contrib_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (56.1 MB)


In [ ]:
import cv2
print("ximgproc available:", hasattr(cv2, "ximgproc"))

ximgproc available: True


In [ ]:
import cv2

# Check if niBlackThreshold exists in OpenCV
if hasattr(cv2.ximgproc, "niBlackThreshold"):
    print("✅ niBlackThreshold is available!")
else:
    print("❌ niBlackThreshold is NOT available!")


✅ niBlackThreshold is available!


In [ ]:
import cv2
print(cv2.__version__)

4.11.0


# Code

## Import and dowload

In [ ]:
import io

In [ ]:
import os
import fitz  # PyMuPDF
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from paddleocr import PaddleOCR
from img2table.document import Image as TableImage
from img2table.ocr import PaddleOCR as Img2TableOCR
from img2table.tables.objects.extraction import ExtractedTable


/usr/local/lib/python3.11/dist-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [ ]:
import torch
import logging

# 🔧 Patch torch.load to bypass weight loading errors
orig_torch_load = torch.load

def torch_wrapper(*args, **kwargs):
    logging.warning("[comfyui-unsafe-torch] `torch.load` is forcibly loading full weights.")
    kwargs['weights_only'] = False
    return orig_torch_load(*args, **kwargs)

torch.load = torch_wrapper

from ultralytics import YOLO

# ✅ Load YOLOv8 table detection model
model = YOLO("/content/sample_data/TableDetection.pt")

# Check if model is loaded successfully
print("✅ YOLOv8 Table Detection Model Loaded Successfully!")



✅ YOLOv8 Table Detection Model Loaded Successfully!


In [ ]:
ocr = PaddleOCR(use_angle_cls=True, lang="vi")
img2table_ocr = Img2TableOCR(lang="vi")


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:14<00:00, 264.82it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer/latin_PP-OCRv3_rec_infer.tar


100%|██████████| 9930/9930 [00:16<00:00, 602.79it/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:15<00:00, 135.60it/s]

[2025/04/03 03:24:57] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_

## Table Processing

In [ ]:

def detect_tables(img):
    results = model(img)  # YOLOv8 table detection
    table_boxes = []

    for r in results:
        for box in r.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            table_boxes.append((x1, y1, x2, y2))

    return table_boxes

def extract_table(img, table_boxes):
    extracted_tables = []

    for box in table_boxes:
        x1, y1, x2, y2 = box
        table_img = img[y1:y2, x1:x2]

        # Convert NumPy array to bytes
        _, buffer = cv2.imencode(".png", table_img)
        table = TableImage(io.BytesIO(buffer)).extract_tables(ocr=img2table_ocr, borderless_tables=True)

        if isinstance(table, list) and table:
            extracted_tables.append(table[0])  # Assuming one table per detection

    return extracted_tables


In [ ]:
import json

def pdf_to_json(folder_path, output_json="extracted_tables.json", zoom=3.0):
    extracted_data = []

    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(".pdf"):
            pdf_path = os.path.join(folder_path, file_name)
            doc = fitz.open(pdf_path)

            document_tables = {
                "document_name": file_name,
                "datatype": "table",
                "extracted_content": []
            }

            for i, page in enumerate(doc):
                # Convert page to image (No storage, process immediately)
                pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom))
                img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)

                # Convert grayscale to BGR if needed
                if img.shape[-1] == 1:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

                # Detect and extract tables
                table_boxes = detect_tables(img)
                extracted_tables = extract_table(img, table_boxes)

                # Append extracted tables
                for table in extracted_tables:
                    if table.df is not None:
                        document_tables["extracted_content"].append(table.df.values.tolist())

            extracted_data.append(document_tables)

    # Save extracted data as JSON
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(extracted_data, f, ensure_ascii=False, indent=4)

    print(f"✅ JSON file saved as {output_json}")

# Example usage
folder_path = "/content/sample_data/TCU"
pdf_to_json(folder_path)


0: 640x480 (no detections), 509.7ms
Speed: 8.0ms preprocess, 509.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 496.6ms
Speed: 6.9ms preprocess, 496.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 484.5ms
Speed: 5.8ms preprocess, 484.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 505.1ms
Speed: 5.2ms preprocess, 505.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 493.1ms
Speed: 6.1ms preprocess, 493.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 493.7ms
Speed: 4.3ms preprocess, 493.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 500.3ms
Speed: 3.9ms preprocess, 500.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 510.0ms
Speed: 5.2ms prepr